In [0]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [0]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [0]:
import numpy as np
import tensorflow as tf
import tflearn
import pandas as pd
import sys
import os
import imageio
import matplotlib.pyplot as plt
import tflearn

import PIL
from PIL import Image

import IPython.display as display
# from IPython.display import display

import pathlib


sys.path.insert(0, "/content/gdrive/My Drive/182_final_project/") # This enables us to import Python libraries in the folder.

root_folder = "/content/gdrive/My Drive/182_final_project/"

tf.reset_default_graph()

In [0]:
# Code to read csv file into colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive2 = GoogleDrive(gauth)
list_paths_work=[]

In [0]:
first = np.load(root_folder + 'first_half_processed.npy', allow_pickle=True)
second = np.load(root_folder + 'second_half_processed_divya.npy', allow_pickle=True)

first = [elem for elem in first if elem != None]
second =  [elem for elem in second if elem != None]

full_data = first + second

first = None
second = None



In [0]:
def center(X): 
  return (X - np.mean(X, axis=0))


In [0]:
# def whiten(X, method='zca'):
#     """
#     Whitens the input matrix X using specified whitening method.
#     Inputs:
#         X:      Input data matrix with data examples along the first dimension
#         method: Whitening method. Must be one of 'zca', 'zca_cor', 'pca',
#                 'pca_cor', or 'cholesky'.
#     """
#     X = X.reshape((-1, np.prod(X.shape[1:])))
#     X_centered = X - np.mean(X, axis=0)
#     Sigma = np.dot(X_centered.T, X_centered) / X_centered.shape[0]
#     W = None
    
#     if method in ['zca', 'pca', 'cholesky']:
#         U, Lambda, _ = np.linalg.svd(Sigma)
#         if method == 'zca':
#             W = np.dot(U, np.dot(np.diag(1.0 / np.sqrt(Lambda + 1e-5)), U.T))
#         elif method =='pca':
#             W = np.dot(np.diag(1.0 / np.sqrt(Lambda + 1e-5)), U.T)
#         elif method == 'cholesky':
#             W = np.linalg.cholesky(np.dot(U, np.dot(np.diag(1.0 / (Lambda + 1e-5)), U.T))).T
#     elif method in ['zca_cor', 'pca_cor']:
#         V_sqrt = np.diag(np.std(X, axis=0))
#         P = np.dot(np.dot(np.linalg.inv(V_sqrt), Sigma), np.linalg.inv(V_sqrt))
#         G, Theta, _ = np.linalg.svd(P)
#         if method == 'zca_cor':
#             W = np.dot(np.dot(G, np.dot(np.diag(1.0 / np.sqrt(Theta + 1e-5)), G.T)), V_sqrt)
#         elif method == 'pca_cor':
#             W = np.dot(np.dot(np.diag(1.0/np.sqrt(Theta + 1e-5)), G.T), V_sqrt)
#     else:
#         raise Exception('Whitening method not found.')

#     return np.dot(X_centered, W.T)

def zca_whitening_matrix(X):
    """
    Function to compute ZCA whitening matrix (aka Mahalanobis whitening).
    INPUT:  X: [M x N] matrix.
        Rows: Variables
        Columns: Observations
    OUTPUT: ZCAMatrix: [M x M] matrix
    """
    # Covariance matrix [column-wise variables]: Sigma = (X-mu)' * (X-mu) / N
    sigma = np.cov(X, rowvar=True) # [M x M]
    # Singular Value Decomposition. X = U * np.diag(S) * V
    U,S,V = np.linalg.svd(sigma)
        # U: [M x M] eigenvectors of sigma.
        # S: [M x 1] eigenvalues of sigma.
        # V: [M x M] transpose of U
    # Whitening constant: prevents division by zero
    epsilon = 1e-5
    # ZCA Whitening matrix: U * Lambda * U'
    ZCAMatrix = np.dot(U, np.dot(np.diag(1.0/np.sqrt(S + epsilon)), U.T)) # [M x M]
    return ZCAMatrix

In [0]:
# class_names = np.array(['Neutral', 'Happy', 'Sad', 'Surprise', 'Fear', 'Disgust', 'Anger', 'Contempt', None, 'Uncertain', 'No-Face'])

In [0]:
neutral = [elem for elem in full_data if int(elem[1]) == 0]
happy = [elem for elem in full_data if int(elem[1]) == 1]
sad = [elem for elem in full_data if int(elem[1]) == 2]
surprise = [elem for elem in full_data if int(elem[1]) == 3]
fear = [elem for elem in full_data if int(elem[1]) == 4]
disgust = [elem for elem in full_data if int(elem[1]) == 5]
anger = [elem for elem in full_data if int(elem[1]) == 6]
contempt = [elem for elem in full_data if int(elem[1]) == 7]
non = [elem for elem in full_data if int(elem[1]) == 8]
uncertain = [elem for elem in full_data if int(elem[1]) == 9]
noface = [elem for elem in full_data if int(elem[1]) == 10]

emotions = [neutral, happy, sad, surprise, fear, disgust, anger, contempt, non, uncertain, noface]

In [0]:
import random

In [0]:
training_data = []
for emotion in emotions:
  if len(emotion) <= 10000: 
    training_data.extend(emotion)
  else:
    subsection = random.sample(emotion, 10000)
    training_data.extend(subsection)
    

In [0]:
print(len(training_data))

train_images = [center(np.array(elem[0])).reshape((32, 32, 3)) for elem in training_data]
train_labels = [elem[1] for elem in training_data]

93417


In [0]:
train_images[0].shape

(32, 32, 3)

In [0]:
# #reads in both halves of the training data and splits each element into 
# #images, labels, and their file paths

# first = np.load(root_folder + 'first_half_processed.npy', allow_pickle=True)
# second = np.load(root_folder + 'second_half_processed_divya.npy', allow_pickle=True)

# first = [elem for elem in first if elem != None]
# second =  [elem for elem in second if elem != None]

# full_data = first + second

# first = None
# second = None

# train_images = [elem[0] for elem in full_data]
# train_labels = [elem[1] for elem in full_data]
# # paths = [elem[2] for elem in full_data]

# full_data = None

# #converts the images into numerical form (32, 32, 3) tensors 
# train_images = [np.array(elem) for elem in train_images]

# #convert images to ints 
# train_labels = [int(elem) for elem in train_labels]

In [0]:
#getting validation data
processed_validation = np.load(root_folder + 'processed_validation.npy', allow_pickle=True)

processed_validation = [elem for elem in processed_validation if elem != None]

val_images = [elem[0] for elem in processed_validation]
val_labels = [elem[1] for elem in processed_validation]
# val_paths = [elem[2] for elem in processed_validation]

processed_validation = None


val_images= [center(np.array(elem)).reshape((32, 32,3)) for elem in val_images]
val_labels = [int(elem) for elem in val_labels]

In [0]:
# class_names = np.array(['Neutral', 'Happy', 'Sad', 'Surprise', 'Fear', 'Disgust', 'Anger', 'Contempt', None, 'Uncertain', 'No-Face'])

In [0]:
'''Train a simple residual network on the CIFAR10 small images dataset.
It gets to 75% validation accuracy in 25 epochs, and 79% after 50 epochs.
(it's still underfitting at that point, though).
'''

from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D, Add
import os
from keras.engine.topology import Layer

# Define the residual block as a new layer
class Residual(Layer):
  def __init__(self, channels_in,kernel,**kwargs):
      super(Residual, self).__init__(**kwargs)
      self.channels_in = channels_in
      self.kernel = kernel

  def call(self, x):
    # the residual block using Keras functional API
    first_layer = Activation("linear", trainable=False)(x)
    x = Conv2D( self.channels_in, self.kernel, padding="same")(first_layer)
    x = Activation("relu")(x)
    x = Conv2D( self.channels_in, self.kernel, padding="same")(x)
    residual = Add()([x, first_layer])
    x = Activation("relu")(residual)
    return x

  def compute_output_shape(self, input_shape):
    return input_shape



In [0]:
class LayerNormalization(Layer):
    def __init__(self, eps=1e-6, **kwargs):
        self.eps = eps
        super(LayerNormalization, self).__init__(**kwargs)
    def build(self, input_shape):
        self.gamma = self.add_weight(name='gamma', shape=input_shape[-1:],
                                     initializer=Ones(), trainable=True)
        self.beta = self.add_weight(name='beta', shape=input_shape[-1:],
                                    initializer=Zeros(), trainable=True)
        super(LayerNormalization, self).build(input_shape)
    def call(self, x):
        mean = K.mean(x, axis=-1, keepdims=True)
        std = K.std(x, axis=-1, keepdims=True)
        return self.gamma * (x - mean) / (std + self.eps) + self.beta
    def compute_output_shape(self, input_shape):
        return input_shape

In [0]:
batch_size = 128
num_classes = 11
epochs = 10
data_augmentation = True
# num_predictions = 20
save_dir = 'root_folder' + 'saved_models'
model_name = 'divabella2'

# The data, split between train and test sets:
# (x_train, y_train), (x_test, y_test) = cifar10.load_data()


x_train = np.array(train_images)
y_train = np.array(train_labels)

x_test = np.array(val_images)
y_test = np.array(val_labels)

# train_images = None
# train_labels = None
# val_images = None
# val_labels = None


print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()


model.add(Conv2D(32, (3, 3), padding='same', input_shape=x_train.shape[1:]))
model.add(Conv2D(32, 3, 3, border_mode='same', input_shape=(32,32,3), activation='relu'))
model.add(Conv2D(32, 3, 3, border_mode='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Conv2D(64, 3, 3, border_mode='same', activation='relu'))
model.add(Conv2D(64, 3, 3, border_mode='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, 3, 3, border_mode='same', activation='relu'))
model.add(Conv2D(128, 3, 3, border_mode='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Conv2D(256, 3, 3, border_mode='same', activation='relu'))
model.add(Conv2D(256, 3, 3, border_mode='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Activation('relu'))
model.add(Residual(256,(3,3)))
model.add(Residual(256,(3,3)))
model.add(Residual(256,(3,3)))
model.add(Residual(256,(3,3)))
model.add(Residual(256,(3,3)))


model.add(Flatten())
model.add(Dense(512))
# model.add(BatchNormalization()) #we are adding this in just to see iteration 2
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        workers=4, steps_per_epoch = 5000)

  # Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = root_folder + 'saved_models/' + model_name
model.save(model_path)
print('Saved trained model at %s ' % model_path)

# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])

x_train shape: (93417, 32, 32, 3)
93417 train samples
5256 test samples


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:37: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(32, 32, 3..., activation="relu", padding="same")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:38: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu", padding="same")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:42: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu", padding="same")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:43: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu", padding="same")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:46: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), activation="relu", padding="same")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:47: UserWar

Using real-time data augmentation.
Epoch 1/10
5000/5000 [==============================] - 323s 65ms/step - loss: 1.6319 - acc: 0.4303 - val_loss: 1.9235 - val_acc: 0.3613
Epoch 2/10
5000/5000 [==============================] - 322s 64ms/step - loss: 1.5774 - acc: 0.4481 - val_loss: 1.9933 - val_acc: 0.3624
Epoch 3/10
5000/5000 [==============================] - 321s 64ms/step - loss: 1.5319 - acc: 0.4630 - val_loss: 2.0373 - val_acc: 0.3621
Epoch 4/10
5000/5000 [==============================] - 320s 64ms/step - loss: 1.4873 - acc: 0.4777 - val_loss: 2.0394 - val_acc: 0.3584
Epoch 5/10
5000/5000 [==============================] - 322s 64ms/step - loss: 1.4462 - acc: 0.4907 - val_loss: 2.1517 - val_acc: 0.3596
Epoch 6/10
5000/5000 [==============================] - 322s 64ms/step - loss: 1.4044 - acc: 0.5049 - val_loss: 2.1242 - val_acc: 0.3525
Epoch 7/10
5000/5000 [==============================] - 324s 65ms/step - loss: 1.3651 - acc: 0.5186 - val_loss: 2.2201 - val_acc: 0.3474
Epoch 

ValueError: ignored

In [0]:
model_path = root_folder + 'saved_models/' + 'best_model'
model.save(model_path)

In [0]:
 # model = Sequential()

# model.add(Conv2D(32, 3, 3, border_mode='same', input_shape=(32,32,3), activation='relu'))
# model.add(Conv2D(32, 3, 3, border_mode='same', activation='relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))

# model.add(Conv2D(64, 3, 3, border_mode='same', activation='relu'))
# model.add(Conv2D(64, 3, 3, border_mode='same', activation='relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Conv2D(32, 3, 3, border_mode='same', activation='relu'))

# model.add()
# model.add(Conv2D(128, 3, 3, border_mode='same', activation='relu'))
# model.add(Conv2D(128, 3, 3, border_mode='same', activation='relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))

# model.add(Conv2D(256, 3, 3, border_mode='same', activation='relu'))
# model.add(Conv2D(256, 3, 3, border_mode='same', activation='relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))

# model.add(Flatten())
# model.add(Dense(256, activation='relu'))
# model.add(Dropout(0.5))

# model.add(Dense(256, activation='relu'))
# model.add(Dropout(0.5))

# model.add(Dense(11))
# model.add(Activation('softmax'))
    
# model.compile(loss='categorical_crossentropy',
#             optimizer=RMSprop(lr=0.0001),
#             metrics=['accuracy'])

In [0]:
# model.fit(np.array(train_images), train_labels, epochs=100, batch_size=5000, validation_data=(np.array(val_images),val_labels))

In [0]:
# model.predict_classes(np.array(val_images))